In [2]:
import os
import json
import pandas
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.externals import joblib
% matplotlib inline

In [3]:
Ned = []
Med = []
NRd = []
MRd = []
fck = []
As = []
bh = []
a = []
xu = []
l0 = []
listdir = os.listdir("../../data_set/buckling")


for file in listdir:
    f = open("../../data_set/buckling/" + file)
    d = json.loads(f.read())
    f.close()

    for o in d:
        Ned.append(o["Ned"])
        #Med.append(o["Med"])
        #NRd.append(o["NRd"])
        #MRd.append(o["MRd"])
        fck.append(o["fck"])
        As.append(o["As"])
        bh.append(o["bh"])
        #a.append(o["a"])
        #xu.append(o["xu"])
        l0.append(o["l0"])

rho = []
for i in range(len(As)):
    rho.append(As[i] / bh[i]**2)
print(max(l0), min(l0))

24359.297950619693 762.303562599071


In [4]:

df = pandas.DataFrame({
        "l0": l0,
        "Ned": Ned,
        #"Med": Med,
        "fck": fck,
        "bh": bh,
        "rho": rho,
    }
    )
print(df.shape)
df = df[df["fck"] == 30]
print(df.shape)

df.sample(n=5).head(5)

(195333, 5)
(66540, 5)


,Ned,bh,fck,l0,rho
110628,2460000,425,30,1664.641867,0.001661
105037,240000,250,30,2435.330199,0.020000
67850,1930000,275,30,996.295962,0.016529
65923,1900000,450,30,2115.789129,0.001481
31969,1410000,675,30,5432.412815,0.000658


In [5]:
# Scale all features by their mean and std-dev to value 0 - 1. This is the correct one.

feat_df = df[["rho", "Ned", "l0"]]
res_df = df[["bh"]]

scale_feat = preprocessing.MinMaxScaler()
scale_feat.fit(feat_df)
print(scale_feat.transform(feat_df))


[[ 0.1088725   0.          0.10326064]
 [ 0.0877827   0.          0.12979905]
 [ 0.0858067   0.          0.16087018]
 ..., 
 [ 0.03070257  0.31010453  0.31365801]
 [ 0.03911252  0.31010453  0.34063085]
 [ 0.04636459  0.31010453  0.36839037]]


In [6]:
df = df.sample(frac=1)
split = int(40e3)

df_train = df[:split]
df_test = df[split:]
df_train.head(5)

print(df_train.shape, df_test.shape)

(40000, 5) (26540, 5)


In [7]:
train_feat = scale_feat.transform(df_train[["rho", "Ned", "l0"]])
train_res = df_train[["bh"]].as_matrix()

test_feat = scale_feat.transform(df_test[["rho", "Ned", "l0"]])
test_res = df_test[["bh"]].as_matrix()

print("Training features:\n rho, Ned, l0\n")
print(train_feat[:5])

print("\nTraining labels:\n bh\n")
print(train_res[:5])
print(train_feat.shape)

Training features:
 rho, Ned, l0

[[ 0.01032626  0.83275261  0.10091735]
 [ 0.00845188  0.65156794  0.13116   ]
 [ 0.35176711  0.09756098  0.20285768]
 [ 0.11393572  0.10801394  0.14047261]
 [ 0.01032626  0.92682927  0.0940902 ]]

Training labels:
 bh

[[575]
 [600]
 [425]
 [400]
 [575]]
(40000, 3)


In [8]:
X = train_feat
y = train_res
y = np.reshape(y, y.shape[0])

In [9]:

m = joblib.load("presentation.pkl")


In [19]:
# Test validation

pred = m.predict(test_feat)

a = df_test[["rho", "Ned", "l0"]].as_matrix()
for i in range(15):
    print("Prediction :", int(pred[i]), "Computed: ", test_res[i][0], "Deviation: %s%%" % (round(1 - (pred[i] / test_res[i])[0], 2) * 100)) 

Prediction : 624 Computed:  625 Deviation: 0.0%
Prediction : 237 Computed:  250 Deviation: 5.0%
Prediction : 381 Computed:  375 Deviation: -2.0%
Prediction : 478 Computed:  475 Deviation: -1.0%
Prediction : 385 Computed:  375 Deviation: -3.0%
Prediction : 444 Computed:  450 Deviation: 1.0%
Prediction : 650 Computed:  650 Deviation: -0.0%
Prediction : 572 Computed:  575 Deviation: 0.0%
Prediction : 569 Computed:  575 Deviation: 1.0%
Prediction : 723 Computed:  725 Deviation: 0.0%
Prediction : 501 Computed:  500 Deviation: -0.0%
Prediction : 523 Computed:  525 Deviation: 0.0%
Prediction : 732 Computed:  725 Deviation: -1.0%
Prediction : 697 Computed:  700 Deviation: 0.0%
Prediction : 676 Computed:  675 Deviation: -0.0%


In [11]:
dev = 0
for i in range(pred.shape[0]):
    dev += abs(1 - (pred[i] / test_res[i])[0])

print("Total accuracy %s%%" % (100 - round(dev / pred.shape[0] * 100, 2)))


Total accuracy 98.85%


In [19]:
joblib.dump(m, 'presentation.pkl') 

['presentation.pkl']

In [18]:
input = {
    "rho": [0.004],
    "Ned": [2500e3],
    "l0": [2e3],
}

for i in input:
    input[i].append(0)

inp = pandas.DataFrame(input)
print(inp.head())

X = scale_feat.transform(inp[["rho", "Ned", "l0"]])
print("\nPrediction:", m.predict(X)[:-1])

h = m.predict(X)[:-1]
ac = h**2
as_ = ac * input["rho"][0]


n = input["Ned"][0] / (ac * 20 + as_ * 435)
ylim = 10.8 / n**0.5

print("\n\nylim", ylim)
print("y", input["l0"][0] / ( h / 3.46 ))

       Ned    l0    rho
0  2500000  2000  0.004
1        0     0  0.000

Prediction: [ 455.22283096]


ylim [ 14.497982]
y [ 15.20134653]
